### Задача (варіант 2)

**Ієрархія:** p = 3, повна, m = (3, 3, 4)

**Метод розрахунку глобальних ваг:** ідеальної точки, ГВБВПА

*Методи аналізу альтернатив рішень на основі ієрархічної моделі критеріїв (МАІ, analytic hierarchy process, AHP)* складаються з наступних чотирьох загальних етапів:

1. Побудова ієрархічної моделі критеріїв, цілей та інших факторів, які впливають на головну ціль прийняття рішення; побудова множини альтернативних варіантів рішень.

2. Отримання суджень експертів щодо парних порівнянь елементів одного рівня ієрархії відносно спільного елементу вищого рівня. Парні порівняння проводяться у вибраній шкалі і за результатами будуються матриці парних порівнянь (МПП), які є обернено симетричними.

3. Математична обробка суджень експертів:
   * розрахунок локальних ваг елементів кожного рівня ієрархії відповідно до батьківських елементів вищого рівня на основі МПП; побудова локальних ранжувань;
   * аналіз узгодженості експертних оцінок;
   * розрахунок глобальних ваг елементів ієрархії відносно головної цілі прийняття рішення, використовуючи методи агрегування; побудова ранжування на основі глобальних ваг.

4. Аналіз чутливості отриманих ранжувань.

Постановка задачі:

**Дано:** 
 * $A = \{A_i | i = 1, \dots, N \}$ - множина альтернативних варіантів рішень;
 * $C = \{C_j | j = 1, \dots, M \}$ - множина критеріїв оцінювання альтернатив;
 * $a_{ij}$ - ненормована вага льтернативи $A_i$ за критерієм $C_j$;
 * $w_j^C$ - вага критерію $C_j, \sum_{j=1}^{M} w_j^C = 1$

**Потрібно:**
 * знайти глобальні ваги $w_i^{глоб}$ альтернатив $A_i, i = 1, \dots, N$

In [1]:
import numpy as np
import math

In [2]:
# number of alternatives
n = 4
# number of criterions C
m = [3,3,4]

#number of layers = P
p=3

Функція для генерування коректної матриці парних порівнянь для мультиплікативної групи

In [10]:
def generate_matched_matrix(n, low=1, high=10):
    """
    Generates matched multiplicational matrix.
    """
    mmp = np.zeros((n, n))
    temp = np.random.randint(low, high, size=n).astype(float)
    powers = np.random.randint(-1, 2, size=n).astype(float)
    mmp[0, :] = np.power(temp, powers)
    j = 0
    for i in range(1, n):
        mmp[i, j] = 1 / mmp[j, i]
    for i in range(1, n):
        for j in range(1, n):
            mmp[i, j] = mmp[i, 0] * mmp[0, j]
    mmp[0][0] = 1
    return mmp

# Data generation

Генеруємо матриці парних порівнянь, та ваги критеріїв.

In [13]:
# put some fixed seed for reproducibility of the results
np.random.seed(44)
# matrices of pair comparisons according to each of 'm' criterion
ddd=[[generate_matched_matrix(m[0])]]
for layer in range(p-1):
    dd=[]
    for i in range(m[layer]):
        dd.append(generate_matched_matrix(m[layer + 1]))
    ddd.append(dd)

# weights of criterions
#w_crit = np.random.randn(m, 1)
ddd

[[array([[1.  , 0.25, 0.5 ],
         [4.  , 1.  , 2.  ],
         [2.  , 0.5 , 1.  ]])], [array([[1.   , 1.   , 8.   ],
         [1.   , 1.   , 8.   ],
         [0.125, 0.125, 1.   ]]), array([[1.        , 1.        , 6.        ],
         [1.        , 1.        , 6.        ],
         [0.16666667, 0.16666667, 1.        ]]), array([[1. , 1. , 0.2],
         [1. , 1. , 0.2],
         [5. , 5. , 1. ]])], [array([[1.        , 3.        , 9.        , 9.        ],
         [0.33333333, 1.        , 3.        , 3.        ],
         [0.11111111, 0.33333333, 1.        , 1.        ],
         [0.11111111, 0.33333333, 1.        , 1.        ]]),
  array([[ 1.  ,  4.  ,  0.2 ,  1.  ],
         [ 0.25,  1.  ,  0.05,  0.25],
         [ 5.  , 20.  ,  1.  ,  5.  ],
         [ 1.  ,  4.  ,  0.2 ,  1.  ]]),
  array([[1.        , 0.33333333, 1.        , 0.11111111],
         [3.        , 1.        , 3.        , 0.33333333],
         [1.        , 0.33333333, 1.        , 0.11111111],
         [9.        ,

Обраховуємо локальні ваги використовуючи підхід EM з попередньої лабораторної роботи

In [14]:
# # generate unnormed local weights of alternatives with respect to all criterions
# # rows are corresponding to alternatives, columns - to criterions
# v = np.random.randn(n, m)

def calculate_local(D):
    """
    D: rank 3 ndarray of shape (m, n, n). m - number of criterions, n - number of alternatives
    Returns (m, n) ndarray where each row represents local weights according to i-th criterion (i in [0, m-1])
    """
    weighcalc = lambda x: math.pow(np.prod(x),1/4.)
    
    res = []
    for layer in ddd:
        layerres = []
        for d in layer:
            v = [weighcalc(di) for di in d]
            w = [vi/np.sum(v) for vi in v]
            layerres.append(w)
        res.append(layerres)
    return res
v = calculate_local(ddd)
v

[[[0.18148095867689784, 0.5133056661466803, 0.30521337517642183]],
 [[0.4524428080730101, 0.4524428080730101, 0.09511438385397966],
  [0.44231202279518583, 0.44231202279518583, 0.11537595440962829],
  [0.18713620050514732, 0.18713620050514732, 0.6257275989897054]],
 [[0.642857142857143,
   0.21428571428571433,
   0.07142857142857144,
   0.07142857142857144],
  [0.13793103448275862,
   0.034482758620689655,
   0.6896551724137931,
   0.13793103448275862],
  [0.07142857142857142,
   0.21428571428571425,
   0.07142857142857142,
   0.6428571428571428]]]

# Methods for finding global normed weights of alternatives

In [15]:

def ideal_point_synthesis(v):
    """
    Computes global weights
    """
    w_glob = [v[0][0]]
    for i in range(1, p):
        r = [np.divide(np.array(vi), np.array(vi).max()) for vi in v[i]]
        vi = [np.sum(vic) for vic in np.multiply(np.array(r).T, w_glob[i - 1])]
        wi_glob = [vij/np.sum(vi) for vij in vi]
        w_glob.append(wi_glob)
    return w_glob
w_g = ideal_point_synthesis(v)
w_g

[[0.18148095867689784, 0.5133056661466803, 0.30521337517642183],
 [0.38356076046023996, 0.38356076046023996, 0.23287847907952017],
 [0.32087543988563444,
  0.1482822537233742,
  0.29837197692374073,
  0.23247032946725066]]

In [18]:
def groupped_concideration_of_binary_alternative_superiority_relation(aij, wjc):
    #gvbvpa
    P = np.identity(n)
    
    for i in range(n):
        for k in range(i, n):
            ri = [aj[i] / (aj[i] + aj[k]) for aj in aij]
            rk = [aj[k] / (aj[i] + aj[k]) for aj in aij]
            wi = np.sum(np.multiply(ri,wjc))
            wk = np.sum(np.multiply(rk,wjc))
            P[i, k] = wi / wk
            P[k, i] = wk / wi
    
    weighcalc = lambda x: math.pow(np.prod(x),1/4.)
    v = [weighcalc(di) for di in P]
    w = [vi / np.sum(v) for vi in v]
    return w

_aij = v[-1]
_wjc = w_g[-2]
w_gvbvpa = groupped_concideration_of_binary_alternative_superiority_relation(_aij, _wjc)
w_gvbvpa

[0.3141295140757584,
 0.19069104795749503,
 0.2532685763609987,
 0.24191086160574785]